# ECG Analysis Example

<div class="alert alert-block alert-info">

This example illustrates how to further process time-series heart rate data extracted from, for instance, ECG data. This includes assigning study conditions, splitting data further into subphases, resampling data, cutting data of all subjects to equal length, normalizing data, rearranging data, as well as computing aggregated results (such as mean and standard error over all subjects for each (sub)phase.

As input, this notebook uses the heart rate processing outputs generated from the ECG processing pipeline as shown in [<code>ECG_Processing_Example.ipynb</code>](ECG_Processing_Example.ipynb)).

</div>

## Setup and Helper Functions

In [ ]:
from pathlib import Path

import re

import pandas as pd
import numpy as np

from fau_colors import cmaps

import biopsykit as bp
from biopsykit.signals.ecg import EcgProcessor

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(
    context="notebook", 
    style="ticks", 
    font="sans-serif",
    palette=palette
)

plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

palette

In [ ]:
# path to import and export example processing results
result_path = Path("./results")

In [ ]:
def display_dict_structure(dict_to_display):
    _display_dict_recursive(dict_to_display)
    
def _display_dict_recursive(dict_to_display):
    if isinstance(dict_to_display, dict):
        display(dict_to_display.keys())
        _display_dict_recursive(list(dict_to_display.values())[0])
    else:
        display("Dataframe shape: {}".format(dict_to_display.shape))
        display(dict_to_display.head())

## Overview

When we collect ECG data from a study, we typically extract heart rate (among others), resulting in time-series heart rate data for each subject. These data are then usually processed further in order to visualize results, perform statistical analyses, etc. How the data is further processed depends on the application. In this example, we will look at two different applications:

1. **Aggregated Results**: This means that we compute aggregations over time-series data. In the end, we will get tabular data with mean values for each subject within one time interval of the study procedure. We compute these *Aggregated Results* when we want to analyze differences, for example, between different study conditions, or between different phases of one study (or both).  
1. **Ensemble Time-series**: This means that we have time-series data where the data of each subject has equal length. In the end, this allows us to concatenate the data of all subjects into a tabular format. We compute these *Ensemble Time-series* data when we want to visualize time-series data of all subjects in one plot by overlaying the data and computing the mean ± standard deviation over all subjects for each time point (a so-called *ensemble plot*).  


Additionally, we will show how to **compute heart rate variability (HRV)** parameters each subject and for each (sub)phase of the recorded data.

## Aggregated Results

For computing *Aggregated Results* the following processing steps will be performed:

1. **Load Data**: Load heart rate processing results per subject and concatenate them into a `SubjectDataDict`, a special nested dictionary structure that contains processed data of all subjects.  
1. **Resample**: Resample heart rate data of all subjects to 1 Hz. This allows to better split the data further into subphases later.  
1. **Normalize Data** (*optional*): Normalize heart rate data with respect to the mean heart rate of a baseline phase. This can allow to better compare heart rate reactions within one study since it removed biases from different resting heart rates.  
1. **Select Phases** (*optional*): If the following steps should not be performed on the data of *all* phases, but only on a *subset* of them (e.g., because you are not interested in the first phase because it is not relevant for your analysis goal, or you only recorded it for normalizing all other phases against it) this subset phases can be specified and further processing will only be performed on this subset.  
1. **Split into Subphases** (*optional*): If the different phases of your study consist of subphases, which you want to aggregate and analyze separately, you can further split the data in the `SubjectDataDict` further into different subphases, which adds a new dictionary level.
1. **Aggregate Results**: Aggregate time-series data by computing the mean heart rate per (sub)phase for each subject. The results will be concatenated into a dataframe.  
1. **Add Conditions** (*optional*): If multiple study conditions were present during the study the study condition can be added as new index level to the dataframe.  

### Load Data

#### Heart Rate Data

Load processed heart rate time-series data of all subjects and concatenate into one dictionary.

In [ ]:
study_data_dict = {}
# or use your own path
ecg_path = bp.example_data.get_ecg_processing_results_path_example()

for file in sorted(ecg_path.glob("hr_result_*.xlsx")):
    subject_id = re.findall("hr_result_(Vp\w+).xlsx", file.name)[0]
    study_data_dict[subject_id] = pd.read_excel(file, sheet_name=None, index_col="time")

Structure of the `SubjectDataDict`:

In [ ]:
display_dict_structure(study_data_dict)

#### Subject Conditions
(will be needed later)

<div class="alert alert-block alert-info">

**Note**: This is only an example, thus, the condition list is manually constructed. Usually, you should import a condition list from a file.
    
</div>

In [ ]:
condition_list = pd.DataFrame(
    ["Control", "Intervention"], 
    columns=["condition"], 
    index=pd.Index(["Vp01", "Vp02"], name="subject")
)
condition_list

### Resample Data

Resample all heart rate values to a sampling rate of 1 Hz.

In [ ]:
dict_result = bp.utils.data_processing.resample_dict_sec(study_data_dict)

Structure of the resampled `SubjectDataDict`:

In [ ]:
display_dict_structure(dict_result)

### Normalize Data

Normalize heart rate data with respect to the mean heart rate of a baseline phase. In this case, the baseline phase is called `"Baseline"`. All heart rate samples are then interpreted as "heart rate change relative to baseline in percent".

In [ ]:
dict_result_norm = bp.utils.data_processing.normalize_to_phase(dict_result, "Baseline")

Structure of the normalized `SubjectDataDict`:

In [ ]:
display_dict_structure(dict_result_norm)

### Select Phases

If desired, select only specific phases, such as only "Intervention", "Stress", and "Recovery" (i.e., drop the "Baseline" phase because it was only used for normalizing all other phases).

In [ ]:
dict_result_norm_selected = bp.utils.data_processing.select_dict_phases(
    dict_result_norm, 
    phases=["Intervention", "Stress", "Recovery"]
)

Structure of the normalized `SubjectDataDict` with only the selected phases:

In [ ]:
display_dict_structure(dict_result_norm_selected)

### Split into Subphases

By splitting the data in the `SubjectDataDict` further into subphases a new index level is added as the most inner dictionary level. You can specify the different subphases in multiple ways (depending on your study protocol):

#### Fixed Lengths, Consecutive Subphases

If all subphases have fixed lengths and subphases are consecutive, i.e., each subphase begins right after the previous one, you can pass a dict with subphase names as keys and subphase *durations* (in seconds) as values.

In [ ]:
dict_result_subph = bp.utils.data_processing.split_dict_into_subphases(
    dict_result, 
    subphases={'Start': 20, 'Middle': 30, 'End': 10}
)

Structure of the `SubjectDataDict` with a new subphase level added:

In [ ]:
display_dict_structure(dict_result_subph)

#### Fixed Lengths, No Consecutive Subphases

If all subphases have fixed lengths, but subphase are *not* consecutive, i.e., do *not* begin right after the previous subphase, you can pass a dict with subphase names as keys and and a tuple with relative start and end *times* (not durations!) (in seconds) as values.

In [ ]:
dict_result_subph = bp.utils.data_processing.split_dict_into_subphases(
    dict_result, 
    subphases={'Start': (0, 10), 'Middle': (20, 40), 'End': (50, 60)}
)

Structure of the `SubjectDataDict` with a new subphase level added:

In [ ]:
display_dict_structure(dict_result_subph)

#### Last Subphase without Fixed Length

If all subphases have fixed length, except the last subphase, which does not have fixed length (e.g. because the last subphase was a phase where feedback was provided to the subject and thus had variable length), but you still want to extract all data from this subphase, you can set the duration of the last subphase to `0`. The duration of this subphase is then inferred from the data by finding the subject with the longest recording. Shorter recordings of other subjects are then also automatically cut to their respective duration.

In [ ]:
dict_result_subph = bp.utils.data_processing.split_dict_into_subphases(
    dict_result, 
    subphases={'Start': 20, 'Middle': 30, 'End': 0}
)

Structure of the `SubjectDataDict` with a new subphase level added:

In [ ]:
display_dict_structure(dict_result_subph)

### Aggregate Results

#### Aggregate Over Phases

In [ ]:
df_result = bp.utils.data_processing.mean_per_subject_dict(
    dict_result, 
    dict_levels=["subject", "phase"], 
    param_name="HR"
)

df_result.head()

#### Aggregate Over Subphases

In [ ]:
df_result_subph = bp.utils.data_processing.mean_per_subject_dict(
    dict_result_subph, 
    dict_levels=["subject", "phase", "subphase"], 
    param_name="HR"
)

df_result_subph.head()

### Add Conditions

In [ ]:
df_result_cond = bp.utils.data_processing.add_subject_conditions(df_result, condition_list)
df_result_cond.head()

In [ ]:
df_result_subph_cond = bp.utils.data_processing.add_subject_conditions(df_result_subph, condition_list)
df_result_subph_cond.head()

### Compute Parameter

The resulting aggregated data can then be further processed and "population averages" can be computed by averaging the heart rate data of each (sub)phase over all subjects, resulting in mean ± standard error values per (sub)phase, and condition (if applicable). The resulting dataframe is a ``MeanSeDataFrame``, a containing mean and standard error of time-series data in a standardized format.

#### Without Conditions

In [ ]:
df_result_mse = bp.utils.data_processing.mean_se_per_phase(df_result)
df_result_mse.head()

In [ ]:
df_result_subph_mse = bp.utils.data_processing.mean_se_per_phase(df_result_subph)
df_result_subph_mse.head()

#### With Conditions

Just calling the same function, but with a different dataframe (where we added the condition as index level).

<div class="alert alert-block alert-info">

**Note**: In this example, the standard error is `NaN` because we only have one subject per condition and can thus not compute the standard error.
    
</div>

In [ ]:
bp.utils.data_processing.mean_se_per_phase(df_result_cond).head()

In [ ]:
bp.utils.data_processing.mean_se_per_phase(df_result_subph_cond).head()

### Plotting

The final data can then either be statistically analyzed, or plotted. Here, we plot the mean and standard error of each phase using the plotting function `biopsykit.protocols.plotting.hr_mean_plot()` from `BioPsyKit`.

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
bp.protocols.plotting.hr_mean_plot(df_result_mse, ax=ax);

If we plot our data with `biopsykit.protocols.plotting.hr_mean_plot()`, and our data consists of different phases, where each phase consist of subphases, the phases will be highlighted in the background for better visibility.

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
bp.protocols.plotting.hr_mean_plot(df_result_subph_mse, ax=ax);

## Ensemble Time-series

For computing *Ensemble Time-series* the following processing steps will be performed:

1. **Load Data**: Load heart rate processing results per subject and concatenate them into a `SubjectDataDict`, a special nested dictionary structure that contains processed data of all subjects.
1. **Resample**: Resample heart rate data of all subjects to 1 Hz. This allows to better split the data further into subphases later.
1. **Normalize Data** (*optional*): Normalize heart rate data with respect to the mean heart rate of a baseline phase. This can allow to better compare heart rate reactions within one study since it removed biases from different resting heart rates.
1. **Select Phases** (*optional*): If the following steps should not be performed on the data of *all* phases, but only on a *subset* of them (e.g., because you are not interested in the first phase because it is not relevant for your analysis goal, or you only recorded it for normalizing all other phases against it) this subset phases can be specified and further processing will only be performed on this subset.
1. **Rearrange Data** The data in form of a `SubjectDataDict` is rearranged into a `StudyDataDict`. A `StudyDataDict` is constructed from a `SubjectDataDict` by swapping outer (subject IDs) and inner (phase names) dictionary levels.
1. **Cut Phases to Equal Length**: If the data don't have equal length per subject (e.g., because the last phase had a flexible duration or was not timed accurately), you can cut the data to the same length in order to concatenate the time-series data afterwards. This is beneficial if you want to overlay time-series data from multiple subjects in an so-called *Ensemble Plot*.
1. **Merge StudyDataDict**: Now that the time-series data of all subjects have equal length within each phase they can be merged into a `MergedStudyDataDict` where the inner level of the nested ``StudyDataDict`` is removed by merging the individual data into one dataframe for each phase.
1. **Add Conditions** (*optional*): If multiple study conditions were present during the study the `StudyDataDict` can further be split by adding a new dictionary level with study conditions to the `StudyDataDict`.

### Load Data

#### Heart Rate Data

In [ ]:
# or use your own path
ecg_path = bp.example_data.get_ecg_processing_results_path_example()

# Load processed heart rate time-series data of all subjects and concatenate into one dict
subject_data_dict = {}
for file in sorted(ecg_path.glob("hr_result_*.xlsx")):
    subject_id = re.findall("hr_result_(Vp\w+).xlsx", file.name)[0]
    subject_data_dict[subject_id] = pd.read_excel(file, sheet_name=None, index_col="time")

Structure of the `SubjectDataDict`:

In [ ]:
display_dict_structure(subject_data_dict)

#### Subject Conditions
(will be needed later)

**Note**: This is only an example, thus, the condition list is manually constructed. Usually, you should import a condition list from a file.

In [ ]:
condition_list = pd.DataFrame(["Control", "Intervention"], columns=["condition"], index=pd.Index(["Vp01", "Vp02"], name="subject"))
condition_list

### Resample Data

Resample all heart rate values to a sampling rate of 1 Hz.

In [ ]:
dict_result = bp.utils.data_processing.resample_dict_sec(subject_data_dict)

Structure of the resampled `SubjectDataDict`:

In [ ]:
display_dict_structure(dict_result)

### Normalize Data

Normalize heart rate data with respect to the mean heart rate of a baseline phase. In this case, the baseline phase is called `"Baseline"`. All heart rate samples are then interpreted as "heart rate change relative to baseline in percent".

In [ ]:
dict_result_norm = bp.utils.data_processing.normalize_to_phase(dict_result, "Baseline")

Structure of the normalized `SubjectDataDict`:

In [ ]:
display_dict_structure(dict_result_norm)

### Select Phases

If desired, select only specific phases, such as only "Intervention", "Stress", and "Recovery" (i.e., drop the "Baseline" phase because it was only used for normalizing all other phases).

In [ ]:
dict_result_norm_selected = bp.utils.data_processing.select_dict_phases(
    dict_result_norm, 
    phases=["Intervention", "Stress", "Recovery"]
)

Structure of the normalized `SubjectDataDict` with only the selected phases:

In [ ]:
display_dict_structure(dict_result_norm_selected)

### Rearrange Data

In [ ]:
dict_study = bp.utils.data_processing.rearrange_subject_data_dict(dict_result_norm)

Structure of the normalized `StudyDataDict`:

In [ ]:
display_dict_structure(dict_study)

### Cut Data

Data for all subjects will be cut to the shortest duration for each phase.

<div class="alert alert-block alert-info">

**Note:** If only a subset of phases should be cut, these phases can be specified using the `phases` parameter.

</div>

In [ ]:
dict_result = bp.utils.data_processing.cut_phases_to_shortest(dict_study)

Structure of the cut `StudyDataDict`:

In [ ]:
display_dict_structure(dict_result)

### Merge `StudyDataDict`

In [ ]:
dict_merged = bp.utils.data_processing.merge_study_data_dict(dict_result)

Structure of the `MergedStudyDataDict`:

In [ ]:
display_dict_structure(dict_merged)

### Add Conditions

In [ ]:
dict_merged_cond = bp.utils.data_processing.split_subject_conditions(dict_merged, condition_list)

Structure of the `MergedStudyDataDict`, split into different conditions:

In [ ]:
display_dict_structure(dict_merged_cond)

### Plotting

After these processing steps we can finally visualize the time-series data ob all subjects in an *Ensemble Plot* where the data ob all subjects is overlaid and each point in the time-series is plotted as mean ± standard error over all subject. For creating Ensemble plots, `BioPsykit` provides the function `biopsykit.protocols.plotting.hr_ensemble_plot`.

<div class="alert alert-block alert-info">

**Note**: These plots does not look as pretty as it should because this is just example data from two subjects. If you want to see plots from "actual" data, just scroll down!

</div>

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
bp.protocols.plotting.hr_ensemble_plot(dict_merged, ax=ax);

If the phases have subphases they can be passed to the plotting function in order to highlight them in the background:

In [ ]:
subphases = {"Start": 20, "Middle": 100, "End": 20}
subphase_dict = {"Baseline": subphases, "Intervention": subphases, "Stress": subphases, "Recovery": subphases}

fig, ax = plt.subplots(figsize=(8,4))
bp.protocols.plotting.hr_ensemble_plot(dict_merged, subphases=subphase_dict, ax=ax);

Load example data from Excel file. The data is already in the format of a `MergedStudyDataDict`, so it can directly be passed to `biopsykit.protocols.plotting.hr_ensemble_plot()`.

In [ ]:
dict_merged_norm = bp.example_data.get_hr_ensemble_sample()

print(bp.utils.datatype_helper.is_merged_study_data_dict(dict_merged_norm))
display_dict_structure(dict_merged_norm)

In [ ]:
subphases = {"Start": 60, "Middle": 240, "End": 0}
subphase_dict = {"Phase1": subphases, "Phase2": subphases, "Phase3": subphases}

fig, ax = plt.subplots(figsize=(8,4))
bp.protocols.plotting.hr_ensemble_plot(dict_merged_norm, subphases=subphase_dict, ax=ax);

## Compute Heart Rate Variability (HRV)

For computing *Heart Rate Variability (HRV)* the following processing steps will be performed:  

1. **Load Data**: Load R-peak data per subject and concatenate them into a `SubjectDataDict`, a special nested dictionary structure that contains processed data of all subjects.  
1. **Select Phases** (*optional*): If the following steps should not be performed on the data of *all* phases, but only on a *subset* of them (e.g., because you are not interested in the first phase because it is not relevant for your analysis goal, or you only recorded it for normalizing all other phases against it) this subset phases can be specified and further processing will only be performed on this subset.  
1. **Split into Subphases** (*optional*): If the different phases of your study consist of subphases, which you want to aggregate and analyze separately, you can further split the data in the `SubjectDataDict` further into different subphases, which adds a new dictionary level.
1. **Compute HRV**: Compute HRV parameters from R-peak data.  
1. **Add Conditions** (*optional*): If multiple study conditions were present during the study the `StudyDataDict` can further be split by adding a new dictionary level with study conditions to the `StudyDataDict`.  

### Load Data

#### R-Peak Data

In [ ]:
# or use your own path
ecg_path = bp.example_data.get_ecg_processing_results_path_example()

# Load processed r-peaks data of all subjects and concatenate into one dict
rpeaks_subject_data_dict = {}
for file in sorted(ecg_path.glob("rpeaks_result_*.xlsx")):
    subject_id = re.findall("rpeaks_result_(Vp\w+).xlsx", file.name)[0]
    rpeaks_subject_data_dict[subject_id] = pd.read_excel(file, sheet_name=None, index_col="time")

In [ ]:
# rename to make it consistent with the other examples
dict_result_rpeaks = rpeaks_subject_data_dict

Structure of the R Peaks `SubjectDataDict`:

In [ ]:
display_dict_structure(dict_result_rpeaks)

#### Subject Conditions
(will be needed later)

<div class="alert alert-block alert-info">

**Note**: This is only an example, thus, the condition list is manually constructed. Usually, you should import a condition list from a file.
    
</div>

In [ ]:
condition_list = pd.DataFrame(["Control", "Intervention"], columns=["condition"], index=pd.Index(["Vp01", "Vp02"], name="subject"))
condition_list

### Select Phases

If desired, select only specific phases, such as only "Intervention", "Stress", and "Recovery".

In [ ]:
dict_result_rpeaks_selected = bp.utils.data_processing.select_dict_phases(
    dict_result_rpeaks, 
    phases=["Intervention", "Stress", "Recovery"]
)

Structure of the `SubjectDataDict` with only the selected phases:

In [ ]:
display_dict_structure(dict_result_rpeaks_selected)

### Split into Subphases

By splitting the data in the `SubjectDataDict` further into subphases a new index level is added as the most inner dictionary level. In this example, we assume that all subphases are consecutive and have fixed length (see the *Aggregated Results* example for further options).

In [ ]:
dict_result_rpeaks_subph = bp.utils.data_processing.split_dict_into_subphases(
    dict_result_rpeaks, 
    subphases={'Start': 20, 'Middle': 30, 'End': 10}
)

Structure of the `SubjectDataDict` with a new subphase level added:

In [ ]:
display_dict_structure(dict_result_rpeaks_subph)

### Compute HRV

Iterate through all subjects and all (sub)phases, call `EcgProcessor.hrv_process()` and concetenate the results into one dataframe.

#### Without Subphases

In [ ]:
dict_hrv_subject = {}
for subject_id, data_dict in dict_result_rpeaks.items():
    list_hrv_phases = []
    for phase, rpeaks in data_dict.items():
        list_hrv_phases.append(EcgProcessor.hrv_process(rpeaks=rpeaks, index=phase, index_name="phase"))
    dict_hrv_subject[subject_id] = pd.concat(list_hrv_phases)

df_hrv = pd.concat(dict_hrv_subject, names=["subject"])
df_hrv.head()

#### With Subphases

Assuming we only want to compute a certain type of HRV measures (in this example, only time-domain measures), we can specify the `hrv_type` in `EcgProcessor.hrv_process()`.

In [ ]:
dict_hrv_subject = {}
for subject_id, data_dict in dict_result_rpeaks_subph.items():
    dict_hrv_phases = {}
    for phase, phase_dict in data_dict.items():
        list_hrv_subphases = []
        for subphase, rpeaks in phase_dict.items():
            list_hrv_subphases.append(
                EcgProcessor.hrv_process(rpeaks=rpeaks, index=subphase, index_name="phase", hrv_types="hrv_time")
            )
        dict_hrv_phases[phase] = pd.concat(list_hrv_subphases)
        
    dict_hrv_subject[subject_id] = pd.concat(dict_hrv_phases, names=["phase"])
    
df_hrv_subph = pd.concat(dict_hrv_subject, names=["subject"])
df_hrv_subph.head()

### Add Conditions

In [ ]:
df_hrv_cond = bp.utils.data_processing.add_subject_conditions(df_hrv, condition_list)
df_hrv_cond.head()

In [ ]:
df_hrv_subph_cond = bp.utils.data_processing.add_subject_conditions(df_hrv_subph, condition_list)
df_hrv_subph_cond.head()

## Note at the End

This example notebook had the purpose to show how to "manually" process such data step by step. However, `BioPsyKit` also offers an easier, object-oriented, way for performing such processing steps in the `biopsykit.protocols` module. For that, you just need to follow these steps:

1. Create a new `Prococol` instance (choose from a set of pre-defined, established laboratory protocols, such as `MIST` or `TSST`, or use the `BaseProtocol`)  
1. Add heart rate data (and R-peak data, if you additionally want to compute HRV), in the form of a `SubjectDataDict` to the `Protocol` instance via `Protocol.add_hr_data()`.  
1. Compute *aggregated results* by calling `Protocol.compute_hr_results()`, *ensemble time-series* by calling `Protocol.compute_hr_ensemble()`, and HRV data by calling `Protocol.compute_hrv_results()`. The single processing steps can be enabled or disabled by passing `True` or `False` as function arguments (have a look at the Documentation!). Additional parameters can be passed to the processing steps by adding them a dictionary and passing it to the function via the `params` argument.  